In [ ]:
import torch
import torch.nn as nn
import random
from model.stylegan.model import Generator, Discriminator
from model.stylegan.dataset import MultiResolutionDataset
from torchvision import transforms, utils
import matplotlib.pyplot as plt

In [ ]:
def make_noise(batch, latent_dim, n_noise, device):
    if n_noise == 1:
        return torch.randn(batch, latent_dim, device=device)

    noises = torch.randn(n_noise, batch, latent_dim, device=device).unbind(0)

    return noises


def mixing_noise(batch, latent_dim, prob, device):
    """有prob的概率产生两个噪音张量元组，1-prob的概率产生单个噪音张量"""
    if prob > 0 and random.random() < prob:
        return make_noise(batch, latent_dim, 2, device)

    else:
        return [make_noise(batch, latent_dim, 1, device)]

In [ ]:
type(make_noise(4, 12, 2, 'cpu'))

In [ ]:
type(make_noise(4, 12, 1, 'cpu'))

In [ ]:
mixing_noise(4, 12, 0.3, 'cpu')

In [ ]:
generator = Generator(
    1024, 512, 8, channel_multiplier=2
).to('cpu')
ckpt = torch.load('./checkpoint/stylegan2-ffhq-config-f.pt', map_location=lambda storage, loc: storage)
generator.load_state_dict(ckpt['g'])
# transform = transforms.Compose(
#     [
#         transforms.RandomHorizontalFlip(),
#         transforms.ToTensor(),
#         transforms.Normalize(
#             (0.5, 0.5, 0.5), (0.5, 0.5, 0.5), inplace=True),
#     ]
# )
# dataset = MultiResolutionDataset('./data/cartoon/lmdb/', transform, 1024)

In [ ]:
noise = mixing_noise(1, 512, 0.9, 'cpu')
fake_img, _ = generator(noise)
plt.imshow(fake_img)
plt.show()